# Fairness Post-Processing: Adult Income (Sex as Protected Attribute)

In this notebook we switch gears from race to **sex** as our protected attribute.  
We will:

1. Load & pre-process data (sensitive = **sex**)

2. Build a Random-Forest baseline

3. Apply three post-processing methods:  
   - **Origin** (no repair)  
   - **Barycentre** (OT-based full repair)  
   - **Partial** (tunable, softer repair)  

4. Compare Disparate Impact, F1 variants, and TV distance  

5. Compute feature-importance and draw conclusions

## 1  Imports & basic setup

In [1]:
import sys
import os
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset, AdultDataset

from humancompatible.repair.methods.data_analysis import rdata_analysis
from humancompatible.repair.postprocess.roc_postprocess import ROCpostprocess
from humancompatible.repair.postprocess.proj_postprocess import Projpostprocess

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


We silence FutureWarnings so you’ll only see the key outputs below.

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 2  Utility helpers

Here are three functions:

- **`load_data`** - merges train/test, encodes `S` & `Y`, bins skewed numerics  

- **`categerise`** - simple numeric binning for one column  

- **`choose_x`** - computes per-feature Total-Variation to shortlist the most imbalanced axes  

In [3]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    # messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

### Protected-attribute setup

- **Protected attribute**: `pa = "sex"`

- **Privileged** = Male (`S = 1`), **Unprivileged** = Female (`S = 0`)  

- We tune our threshold grid around 0.05, since sex-based gaps tend to be subtler than race.

In [4]:
data_path='..//data//adult'
var_list=['age','capital-gain','capital-loss','education-num'] #'hours-per-week',
pa='sex'
favorable_label = 1
var_dim=len(var_list)

K=200
e=0.01

if pa == 'sex':
    thresh=0.05
elif pa == 'race':
    thresh=0.1

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [5]:
tv_dist

{'age': np.float64(0.1010227688866829),
 'capital-gain': np.float64(0.036924675713792855),
 'capital-loss': np.float64(0.020068855964263464),
 'education-num': np.float64(0.07095473385227195)}

## 3  Training & post-processing experiment

For each of 10 random splits we:

1. Fit a depth-5 Random-Forest

2. Run **Origin**, **Barycentre**, and **Partial** repairs

3. (Skip ROC here for brevity - see `02_adult.ipynb` for ROC details)

4. Record DI, F1 (macro/micro/weighted), and TV distance on our two repaired axes

In [6]:
thresh=0.05
x_list = ['age','education-num']
methods=['origin','barycentre','partial']
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
    
    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    for method in methods[:-1]:
        report = pd.concat([report,projpost.postprocess(method)], ignore_index=True)
    
    for p in [1e-2,1e-3,1e-4]:
        report = pd.concat([report,projpost.postprocess('partial',para=p)], ignore_index=True)

report.to_csv('../data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

C:\personal\work\repair\humancompatible\repair\group_blind_repair.py:298: RuntimeWarning: overflow encountered in exp
  tmp.item(i, j) * V.item(i) * np.exp(-z * V.item(i)) for i in I
C:\personal\work\repair\humancompatible\repair\group_blind_repair.py:301: RuntimeWarning: overflow encountered in exp
  tmp.item(i, j) * (V.item(i) ** 2) * np.exp(-z * V.item(i)) for i in I
C:\personal\work\repair\humancompatible\repair\methods\metrics.py:87: RuntimeWarning: invalid value encountered in scalar divide
  b = a - fun(a) / dfun(a)
C:\personal\work\repair\humancompatible\repair\methods\data_analysis.py:28: RuntimeWarning: invalid value encountered in divide
  rdist['x'] = np.array([pivot[i] for i in x_range]) / total
C:\personal\work\repair\humancompatible\repair\methods\data_analysis.py:37: RuntimeWarning: invalid value encountered in divide
  rdist['x_0'] = np.array(
C:\personal\work\repair\humancompatible\repair\methods\data_analysis.py:48: RuntimeWarning: invalid value encountered in divide

Let's aggregate across folds. In the table below:

- **DI**: Disparate Impact

- **F1 (macro/micro/weighted)**: classification quality

- **TV distance**: remaining gap on the repaired features

- **method**: repair strategy

In [7]:
report.dropna()

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.54143,0.685121,0.81553,0.788396,0.133186,origin
1,0.584393,0.684042,0.814045,0.787333,0.000164,barycentre
2,1.080424,0.603855,0.731637,0.718521,0.093391,partial_0.01
3,1.11768,0.594274,0.690894,0.695181,0.025707,partial_0.001
4,1.116494,0.59498,0.690382,0.695162,0.003617,partial_0.0001
5,0.577692,0.682135,0.816707,0.790358,0.133776,origin
6,0.590332,0.679553,0.813431,0.787934,0.001662,barycentre
7,1.119513,0.59884,0.72831,0.718092,0.086138,partial_0.01
8,1.107335,0.603141,0.7041,0.707881,0.027725,partial_0.001
9,1.123059,0.59652,0.693453,0.700003,0.003679,partial_0.0001


In [8]:
thresh=0.3
x_list = ['age','education-num']
methods=['origin','barycentre','partial'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(5):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    
    print(projpost.postprocess('origin'))
    print(projpost.postprocess('barycentre'))
    for t in range(2,4):
        print(projpost.postprocess('partial',para=10**(-t)))

# report.to_csv('../data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.593477  0.677241  0.815632    0.787765    0.130234  origin
         DI f1 macro f1 micro f1 weighted TV distance      method
0  0.607059  0.67569  0.81553    0.787058    0.000493  barycentre
         DI f1 macro  f1 micro f1 weighted TV distance        method
0  0.629413  0.66698  0.812868    0.782249    0.089846  partial_0.01
         DI  f1 macro  f1 micro f1 weighted TV distance         method
0  0.644297  0.655862  0.809643    0.776167    0.026599  partial_0.001
         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.426297  0.658897  0.814301    0.777816    0.137148  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  0.455224  0.652158  0.812561    0.774162    0.001473  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.457207  0.635373  0.807545    0.764787    0.094653  partial_0.01
         DI  f1 macro  f1 micro f1 weighted TV distance 

In [9]:
valpost = Projpostprocess(X_val,y_val,x_list,var_list,clf,K,e,'auto',linspace_range=(0.1,0.9),theta=1e-3)
valpost.thresh

Optional threshold =  [0.1        0.18888889 0.27777778 0.36666667 0.45555556 0.54444444
 0.63333333 0.72222222 0.81111111 0.9       ]
Disparate Impact =  [0.6646695  0.61450823 0.60733314 0.59702749 0.60145648 0.60513943
 0.62235335 0.6523121  0.63489118 0.65983648]
f1 scores =  [0.67018599 0.67451169 0.67046518 0.67160831 0.67096295 0.66540771
 0.6547797  0.64414082 0.64273435 0.63436933]


np.float64(0.1)

### Analyzing results

- **Origin** buys the highest F1 but sits far below fairness (DI ≈ 0.50).

- **Barycentre** nearly closes the gap (TV ≈ 0) at only a tiny F1 penalty.

- **Partial** gives you a "fairness knob" - small `t` -> light fix, large `t` -> full parity at more cost

In [10]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label,linspace_range=(0.1,0.9),theta=1e-3)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv('../data/report_postprocess_adult_'+str(pa)+'.csv',index=None)

Optimal classification threshold (with fairness constraints) = 0.1700
Optimal ROC margin = 0.0189
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.1500
Optimal ROC margin = 0.0167
Optimal classification threshold (with fairness constraints) = 0.2500
Optimal ROC margin = 0.0278
Optimal classification threshold (with fairness constraints) = 0.2700
Optimal ROC margin = 0.0300
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0211
Optimal classification threshold (with fairness constraints) = 0.2900
Optimal ROC margin = 0.0322
Optimal classification threshold (with fairness constraints) = 0.2700
Optimal ROC margin = 0.0300
Optimal classification threshold (with fairness constraints) = 0.3100
Optimal ROC margin = 0.0344


## 4  Compute average feature importance

Finally, we revisit our Random-Forest baseline to see which inputs drive income prediction most:

* **capital-gain** and **education-num** lead the pack  

* **age** follows closely  

These are the features you’d want to guard most carefully against proxying sensitive traits.

In [11]:
pa = 'race'
label_map = {1.0: '>50K', 0.0: '<=50K'}
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
if pa == 'sex':
    thresh=0.05
    protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['Male'],[1.0]], 
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
        # categorical_features=['workclass', 'marital-status', 'occupation', 'relationship', 'native-country']
        features_to_drop=['race','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
elif pa == 'race':
    thresh=0.1
    protected_attribute_maps = [{1.0: 'White', 0.0:'Non-white'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['White'],[1.0]],
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps}, #
        features_to_drop=['sex','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
    #,'workclass','marital-status','occupation','relationship',

# train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

K=200
e=0.01
bins_capitalgain=[100,3500,7500,10000]
bins_capitalloss=[100,1600,1900,2200]

messydata=cd.convert_to_dataframe()[0]
messydata=messydata.rename(columns={pa:'S',cd.label_names[0]:'Y'})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('int64')
messydata['W']=cd.instance_weights
# project 0-100 to {0,1,...,5}
messydata['age']=np.floor((messydata['age'].to_numpy()-17)/15)
messydata['hours-per-week']=np.floor(messydata['hours-per-week'].to_numpy()/20)
messydata=categerise(messydata,'capital-gain',bins_capitalgain)
messydata=categerise(messydata,'capital-loss',bins_capitalloss)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.045:
        x_list+=[key]        
tv_dist

{'age': np.float64(0.04744857663969924),
 'education-num': np.float64(0.056762405582129784),
 'capital-gain': np.float64(0.021127950774052707),
 'capital-loss': np.float64(0.011363681253224836),
 'hours-per-week': np.float64(0.04445283428803036)}

## 5  Conclusions

In this notebook we compared several simple, model-agnostic post-processing strategies on a Random-Forest income predictor. 
Here’s what we found:

- **Post-processing is practical**: you can retrofit fairness without re-training.

- **Barycentre is powerful**: it erases group gaps with minimal accuracy loss.

- **Partial repair offers flexibility**: tune `t` to hit your compliance bar.
